<a href="https://colab.research.google.com/github/Ron-Rocks/Estimator/blob/master/Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np 


# Pre-processing the data from the Mnist dataset

In [0]:
mnist = tf.keras.datasets.mnist
(xTrain,yTrain),(xTest,yTest) = mnist.load_data()

xTrain = (xTrain.reshape(-1,784))/255
xTest = (xTest.reshape(-1,784))/255
 
print(yTrain)

[5 0 4 ... 5 6 8]


# From Keras

In [0]:
inputLayer = keras.Input(shape = (784,),name="x")
x = layers.Dense(64,activation = "relu")(inputLayer)
x =layers.Dense(128,activation = "relu")(x)
x = layers.Dense(64,activation = "relu")(x)
outputLayer = layers.Dense(10,activation = "softmax")(x)

Kerclassifier = keras.Model(inputs = inputLayer,outputs = outputLayer)


Kerclassifier.compile(loss = "sparse_categorical_crossentropy",optimizer = "adam",metrics = ["accuracy"])
Kerclassifier.fit(xTrain,yTrain,epochs = 5,validation_data = (xTest,yTest))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 71us/sample - loss: 0.2622 - acc: 0.9216 - val_loss: 0.1529 - val_acc: 0.9536
Epoch 2/5
60000/60000 [==============================] - 4s 72us/sample - loss: 0.1158 - acc: 0.9648 - val_loss: 0.1147 - val_acc: 0.9644
Epoch 3/5
60000/60000 [==============================] - 4s 71us/sample - loss: 0.0864 - acc: 0.9731 - val_loss: 0.0961 - val_acc: 0.9706
Epoch 4/5
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0690 - acc: 0.9782 - val_loss: 0.0901 - val_acc: 0.9735
Epoch 5/5
60000/60000 [==============================] - 4s 70us/sample - loss: 0.0565 - acc: 0.9822 - val_loss: 0.1018 - val_acc: 0.9718


# Now With Estimator

tf.estimator is a High level api for making Neural Nets.

Either we can use the in-built estimators like:

1.DNNClassifier (used here)

2.LinearClassifier

3.LinearRegressor
.......etc

>Estimator are highly useful on large Datasets that can export a model for production 

>because this is the API which works well with Cloud ML Engines



In [0]:
# converting label data type to integer
yTrain = yTrain.astype(int)
yTest = yTest.astype(int)

# The only features column that our dataset has and naming it as "x" with shape(28,28) which is the shape of mnist data
features_column = [tf.feature_column.numeric_column("x",shape = [784])]

#Actually Making the model 
# as DNNClassifier(feature_columns: the column of data we need to pass which is defined just above
#                 model_dir : this is where our model will get saved
#                 n_classes : this is the number of classes we need to catagorize our data into (in our case - 10 digits)
#                 hidden_units : it is the number of dense layers and number of neurons in each
classifier = tf.estimator.DNNClassifier(feature_columns=features_column,model_dir="model/MnistModel",n_classes=10,hidden_units=[64,128,64],activation_fn=tf.nn.relu)

# creating the input function which returns the image data and the labels
#epochs = 1 means it will go through the data only once during testing
trainInputFn = tf.estimator.inputs.numpy_input_fn(x = {"x":xTrain},y = yTrain,num_epochs=None,shuffle=True)
testInputFn = tf.estimator.inputs.numpy_input_fn(x = {"x":xTest},y = yTest,num_epochs=1,shuffle=False)

#Training the model by setting the number of steps the model should take
classifier.train(trainInputFn,steps=300)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/MnistModel', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa66b2aa748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Vari

# Now Evaluating both Models

In [0]:
# Evaluating the model using testFunction as the inputs
EstEval = classifier.evaluate(input_fn=testInputFn)
# Now, evaluating the Keras Model 
KerEval = Kerclassifier.evaluate(xTest,yTest)


print("Results with Keras :  accuracy : ",KerEval[1]," loss :  ",KerEval[0])
print("Results with Estimator : ",EstEval)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-08T11:04:33Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/MnistModel/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-12-08-11:04:33
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.9534, average_loss = 0.15263298, global_step = 1000, loss = 19.320631
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: model/MnistModel/model.ckpt-1000
10000/10000 [==============================] - 0s 29us/sample - loss: 0.1018 - acc: 0.9718
Results with Keras :  accuracy :  0.9718  loss :   0.10176746470418294
Results with Estimator :  {'accuracy': 0.9534, 'average_loss': 0.15263298, 'loss': 19.320631, 'global_step': 1000}


# Converting keras to Estimator

In [0]:
# Model  Keras ----> Estimator
convertedModel = tf.keras.estimator.model_to_estimator(Kerclassifier)

ConvEval = convertedModel.evaluate(input_fn = testInputFn)
print("Converted model : ",ConvEval)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmplv7xx6a3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training